In [1]:
import sys
sys.path.append('../')
import pickle
import re
import glob
import os

import time

import theano
import theano.tensor as T
import numpy as np
import pandas as p
import lasagne as nn

from utils import hms, architecture_string, get_img_ids_from_iter

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


In [2]:
!gcc --version

gcc (Ubuntu 4.8.5-4ubuntu8~16.04.1) 4.8.5
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
!which nvcc

/usr/local/cuda-7.0/bin/nvcc


In [4]:
!python --version

Python 2.7.16 :: Anaconda, Inc.


In [4]:
!git clone https://github.com/Theano/Theano
%cd Theano
!git checkout 9a653e3e91c0e38b6643e4452199931e792a24a2
!pip install .
%cd ..

fatal: destination path 'Theano' already exists and is not an empty directory.
/users/avanti/kaggle_diabetic_retinopathy/notebooks/Theano
HEAD is now at 9a653e3... Merge pull request #3150 from caglar/theano_shared_zero2
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Processing /users/avanti/kaggle_diabetic_retinopathy/notebooks/Theano
  Stored in directory: /tmp/pip-ephem-wheel-cache-88DvGn/wheels/2e/80/ce/e61c44e4fc9a975b404057b2fe01224d2ea79dec59f36d64ec
Successfully built Theano
  Found existing installation: Theano 0.7.0
    Uninstalling Theano-0.7.0:
      Successfully uninstalled Theano-0.7.0
/users/avanti/kaggle_diabetic_retinopathy/notebooks


In [5]:
!git clone https://github.com/Lasagne/Lasagne
%cd Lasagne
!git checkout cf1a23c21666fc0225a05d284134b255e3613335
!pip install .
%cd ..

fatal: destination path 'Lasagne' already exists and is not an empty directory.
/users/avanti/kaggle_diabetic_retinopathy/notebooks/Lasagne
HEAD is now at cf1a23c... Merge pull request #278 from skaae/add_embedding_doc
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Processing /users/avanti/kaggle_diabetic_retinopathy/notebooks/Lasagne
  Stored in directory: /tmp/pip-ephem-wheel-cache-NQqxkB/wheels/41/17/42/bed688d24ee7c4d3f5638aa526295c91eba55d8d3aec20e9b1
Successfully built Lasagne
  Found existing installation: Lasagne 0.1.dev0
    Uninstalling Lasagne-0.1.dev0:
      Successfully uninstalled Lasagne-0.1.dev0
/users/avanti/kaggle_diabetic_retinopathy/notebooks


In [5]:
!conda list

# packages in environment at /users/avanti/anaconda3/envs/py2diabeticretinopathy:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
attrs                     19.1.0                   py27_1  
backports                 1.0                        py_2  
backports.shutil_get_terminal_size 1.0.0                    py27_2  
backports_abc             0.5                        py_0  
blas                      1.0                         mkl  
bleach                    3.1.0                    py27_0  
ca-certificates           2019.5.15                     0  
cairo                     1.12.18                       0  
certifi                   2019.6.16                py27_1  
configparser              3.7.4                    py27_0  
cycler                    0.10.0                   py27_0  
cython                    0.17                     pypi_0    pypi
dateutil                  2.4.1                    py2

In [6]:
%pylab inline

rcParams['figure.figsize'] = 16, 6

Populating the interactive namespace from numpy and matplotlib


In [7]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

First we load the dump of the trained network.

In [8]:
dump_path = '../dumps/2015_07_17_123003.pkl'

In [9]:
model_data = pickle.load(open(dump_path, 'r'))

In [10]:
# Let's set the in and output layers to some local vars.
l_out = model_data['l_out']
l_ins = model_data['l_ins']

Some info about the architecture of the model:

In [11]:
model_arch = architecture_string(model_data['l_out'])

num_params = nn.layers.count_params(l_out)
model_arch += "\nNumber of parameters: %d.\n\n" % num_params
print(model_arch)

   0  InputLayer                   (64, 3, 512, 512)  
   1  Conv2DDNNLayer               (64, 32, 256, 256)  7 //2
   2  MaxPool2DDNNLayer            (64, 32, 127, 127)  3 //2
   3  Conv2DDNNLayer               (64, 32, 127, 127)  3 //1
   4  Conv2DDNNLayer               (64, 32, 127, 127)  3 //1
   5  MaxPool2DDNNLayer            (64, 32, 63, 63)  3 //2
   6  Conv2DDNNLayer               (64, 64, 63, 63)  3 //1
   7  Conv2DDNNLayer               (64, 64, 63, 63)  3 //1
   8  MaxPool2DDNNLayer            (64, 64, 31, 31)  3 //2
   9  Conv2DDNNLayer               (64, 128, 31, 31)  3 //1
  10  Conv2DDNNLayer               (64, 128, 31, 31)  3 //1
  11  Conv2DDNNLayer               (64, 128, 31, 31)  3 //1
  12  Conv2DDNNLayer               (64, 128, 31, 31)  3 //1
  13  MaxPool2DDNNLayer            (64, 128, 15, 15)  3 //2
  14  Conv2DDNNLayer               (64, 256, 15, 15)  3 //1
  15  Conv2DDNNLayer               (64, 256, 15, 15)  3 //1
  16  Conv2DDNNLayer               (64, 256, 

/users/avanti/anaconda3/envs/py2diabeticretinopathy/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


Setting up some Theano / Lasagne things to get some predictions.

In [12]:
chunk_size = model_data['chunk_size'] * 2
batch_size = model_data['batch_size']

print "Batch size: %i." % batch_size
print "Chunk size: %i." % chunk_size

Batch size: 64.
Chunk size: 256.


In [13]:
output = nn.layers.get_output(l_out, deterministic=True)
nondeterministic_output = nn.layers.get_output(l_out, deterministic=False)
input_ndims = [len(nn.layers.get_output_shape(l_in))
               for l_in in l_ins]
xs_shared = [nn.utils.shared_empty(dim=ndim)
             for ndim in input_ndims]
idx = T.lscalar('idx')

givens = {}
for l_in, x_shared in zip(l_ins, xs_shared):
    givens[l_in.input_var] = x_shared[idx * batch_size:(idx + 1) * batch_size]

compute_output = theano.function(
    [idx],
    output,
    givens=givens,
    on_unused_input='ignore'
)

compute_nondeterministic_output = theano.function(
    [idx],
    nondeterministic_output,
    givens=givens,
    on_unused_input='ignore'
)

In [14]:
compute_output2 = theano.function(
    [x.input_var for x in l_ins],
    output,
    on_unused_input='ignore')

In [15]:
compute_nondeterministic_output2 = theano.function(
    [x.input_var for x in l_ins],
    nondeterministic_output,
    on_unused_input='ignore')

In [16]:
train_labels = p.read_csv(os.path.join('/srv/scratch/avanti/diabetic_retinopathy/trainLabels.csv'))

In [17]:
train_labels

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1


In [18]:
paired_transfos=True

The validation set is a specific subset of the training set, so load the train labels

In [19]:
image_to_label = dict(zip(train_labels.image, train_labels.level))
new_dataloader_params = model_data['data_loader_params']
train_image_ids = set(new_dataloader_params['images_train_0'])
# Get validation set ids
all_patient_ids = sorted(set(get_img_ids_from_iter(train_labels.image)))
valid_patient_ids = [patient_id for patient_id in all_patient_ids
                     if patient_id not in train_image_ids]

In [20]:
valid_patient_ids[:20]

[19,
 52,
 58,
 60,
 74,
 81,
 122,
 164,
 194,
 195,
 218,
 224,
 239,
 242,
 260,
 311,
 314,
 343,
 358,
 360]

In [21]:
valid_patient_ids_labels = [image_to_label[str(patient_id)+"_"+side]
                            for patient_id in valid_patient_ids
                            for side in ["left","right"]]

In [22]:
len(valid_patient_ids_labels)

3514

In [23]:
valid_labels_fh = open("valid_labels.txt",'w')
for patient_id in valid_patient_ids:
    for side in ["left", "right"]:
        patient_id_and_side = str(patient_id)+"_"+side
        valid_labels_fh.write(
            patient_id_and_side+"\t"+
            "\t".join(str(image_to_label[patient_id_and_side])+"\n"))
valid_labels_fh.close()
!gzip -f valid_labels.txt

In [24]:
num_chunks = int(np.ceil((2 * len(valid_patient_ids)) / float(chunk_size)))

In [25]:
num_chunks

14

In [26]:
# Where all the images are located: 
# it looks for [img_dir]/[patient_id]_[left or right].jpeg
img_dir = '/srv/scratch/avanti/diabetic_retinopathy/unzipped_train_ds2_crop/'

Using the DataLoader to set up the parameters, you could replace it with something much simpler.

In [27]:
from generators import DataLoader

In [28]:
data_loader = DataLoader()
#new_dataloader_params.update({'images_test': patient_ids})
#new_dataloader_params.update({'labels_test': train_labels.level.values})
new_dataloader_params.update({'prefix_train': img_dir})
data_loader.set_params(new_dataloader_params)

In [29]:
def do_pred(test_gen, deterministic):
    outputs = []

    for e, (xs_chunk, chunk_shape, chunk_length) in enumerate(test_gen()):
        num_batches_chunk = int(np.ceil(chunk_length / float(batch_size)))

        print "Chunk %i/%i" % (e + 1, num_chunks)

        print "  load data onto GPU"
        for x_shared, x_chunk in zip(xs_shared, xs_chunk):
            x_shared.set_value(x_chunk)

        print "  compute output in batches"
        outputs_chunk = []
        for b in xrange(num_batches_chunk):
            if (deterministic):
                out = compute_output(b)
            else:
                out = compute_nondeterministic_output(b)
            outputs_chunk.append(out)

        outputs_chunk = np.vstack(outputs_chunk)
        outputs_chunk = outputs_chunk[:chunk_length]

        outputs.append(outputs_chunk)

    return np.vstack(outputs), xs_chunk

In [30]:
from generators import patches_gen_fixed_pairs


def create_fixed_gen(data_loader, images, chunk_size,
                     prefix_train, prefix_test,
                     transfo_params=None,
                     paired_transfos=paired_transfos):

    if not transfo_params:
        raise ValueError("Need transfo_params for gen!")

    gen = patches_gen_fixed_pairs(
            images, p_x=data_loader.p_x,
            p_y=data_loader.p_y,
            num_channels=data_loader.num_channels,
            chunk_size=chunk_size,
            prefix_train=prefix_train,
            prefix_test=prefix_test,
            transfo_params=transfo_params,
            paired_transfos=paired_transfos)

    def fixed_gen():
        for chunk_x, chunk_dim, chunk_shape, chunk_length in gen:
            yield [(chunk_x - data_loader.zmuv_mean) /
                   (0.05 + data_loader.zmuv_std),
                   chunk_dim], chunk_shape, chunk_length
    return fixed_gen()

In [31]:
no_transfo_params = model_data['data_loader_params']['no_transfo_params']

transfo_param_sets = [
    #{'flip': False,
    # 'rotation_before_resize': True,
    # 'rotation_range': (0,1)},
    #{'flip': True,
    # 'flip_prob': 1.0,
    # 'rotation_before_resize': True,
    # 'rotation_range': (0,1)},
    #{'flip': False,
    # 'rotation_before_resize': True,
    # 'rotation_range': (90,91)},
    #{'flip': True,
    # 'flip_prob': 1.0,
    # 'rotation_before_resize': True,
    # 'rotation_range': (90,91)},
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (180,181)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (180,181)},
    {'flip': False,
     'rotation_before_resize': True,
     'rotation_range': (270,271)},
    {'flip': True,
     'flip_prob': 1.0,
     'rotation_before_resize': True,
     'rotation_range': (270,271)},
]

def print_predictions(output_fh, pred_outputs, patient_ids):
    for i,patient_id in enumerate(patient_ids):
        for side in ['left', 'right']:
            if (side=='left'):
                pred = pred_outputs[i*2]
            else:
                pred = pred_outputs[i*2 + 1]
            output_fh.write(str(patient_id)+"_"+side+"\t"
                            +"\t".join([str(x) for x in pred])+"\n")

for transfo_params in transfo_param_sets:
    transfo_string_summary = (
        "flip-"+str(transfo_params['flip'])
        +"_rotamt-"+str(transfo_params['rotation_range'][0]))
    #transfo_string_summary="notransfo"
    if (os.path.exists(transfo_string_summary)==False):
        os.mkdir(transfo_string_summary)
    full_transfo_params = {}
    full_transfo_params.update(no_transfo_params)
    full_transfo_params.update(transfo_params)
    data_generator = lambda: create_fixed_gen(
                        data_loader=data_loader,
                        images=valid_patient_ids,
                        chunk_size=chunk_size,
                        prefix_train=img_dir,
                        prefix_test=img_dir,
                        transfo_params=full_transfo_params,
                        paired_transfos=paired_transfos)
    print("Doing deterministic predictions")
    sys.stdout.flush()
    pred_outputs, chunk_orig = do_pred(test_gen=data_generator,
                                       deterministic=True)
    """output_file = transfo_string_summary+"/deterministic_preds.txt"
    output_fh = open(output_file,'w')
    print_predictions(output_fh=output_fh,
                      pred_outputs=pred_outputs,
                      patient_ids=valid_patient_ids)
    output_fh.close()"""
    for nondeterministic_run in range(100):
        print("nondet run",nondeterministic_run)
        sys.stdout.flush()
        pred_outputs, chunk_orig = do_pred(test_gen=data_generator,
                                           deterministic=False)
        """output_file = (transfo_string_summary+
                       "/nondeterministic_preds_"
                       +str(nondeterministic_run)+".txt")
        output_fh = open(output_file,'w')
        print_predictions(output_fh=output_fh,
                          pred_outputs=pred_outputs,
                          patient_ids=valid_patient_ids)"""
        output_fh.close()

Doing deterministic predictions
Chunk 1/14
  load data onto GPU
  compute output in batches


KeyboardInterrupt: 

In [32]:
thegen = data_generator()

In [ ]:
dat = thegen.next()
print("hi")

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(dat[0][0][0].transpose((2,1,0))[:,:,0])

In [58]:
dat[1]

array([[   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [   3.,  512.,  512.],
       [  